<a href="https://colab.research.google.com/github/SinnottKayleigh/B2B-Sales-Algos/blob/main/Relationship_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00


In [5]:
class Company:
    def __init__(self, name, ticker_symbol):
        self.name = name
        self.ticker_symbol = ticker_symbol
        self.subsidiaries = []
        self.financial_metrics = {
            'annual_revenue': 0,
            'credit_risk_score': 0,
            'fx_exposure': 0,
            'conversion_likelihood': 0
        }

class Subsidiary:
    def __init__(self, name, parent_company):
        self.name = name
        self.parent_company = parent_company
        self.financial_metrics = {
            'annual_revenue': 0,
            'credit_risk_score': 0,
            'fx_exposure': 0,
            'conversion_likelihood': 0
        }

In [7]:
import networkx as nx
import pandas as pd
import numpy as np

class SubsidiaryNetworkAnalyzer:
    def __init__(self):
        self.graph = nx.DiGraph()

    def add_company(self, company):
        self.graph.add_node(company.name, type='parent', metrics=company.financial_metrics)

    def add_subsidiary(self, subsidiary, parent_name):
        self.graph.add_node(subsidiary.name, type='subsidiary', metrics=subsidiary.financial_metrics)
        self.graph.add_edge(parent_name, subsidiary.name)

    def calculate_network_metrics(self):
        degree_centrality = nx.degree_centrality(self.graph)
        betweenness_centrality = nx.betweenness_centrality(self.graph)

        conversion_scores = {}
        for node in self.graph.nodes():
            node_metrics = self.graph.nodes[node].get('metrics', {})

            conversion_score = (
                0.3 * node_metrics.get('annual_revenue', 0) / 1000000 +  # Revenue impact
                0.3 * (1 - node_metrics.get('credit_risk_score', 0)) +   # Lower risk
                0.2 * (1 - abs(node_metrics.get('fx_exposure', 0))) +    # Lower FX volatility
                0.2 * degree_centrality.get(node, 0)                     # Network centrality
            )

            conversion_scores[node] = conversion_score

        return {
            'degree_centrality': degree_centrality,
            'betweenness_centrality': betweenness_centrality,
            'conversion_likelihood': conversion_scores
        }

    def identify_high_potential_subsidiaries(self, top_n=5):
        network_metrics = self.calculate_network_metrics()

        ranked_subsidiaries = sorted(
            network_metrics['conversion_likelihood'].items(),
            key=lambda x: x[1],
            reverse=True
        )

        return ranked_subsidiaries[:top_n]

In [8]:
class RiskAnalyzer:
    @staticmethod
    def calculate_fx_risk(subsidiaries):
        fx_exposures = [sub.financial_metrics['fx_exposure'] for sub in subsidiaries]
        return {
            'average_fx_exposure': np.mean(fx_exposures),
            'fx_risk_volatility': np.std(fx_exposures)
        }

    @staticmethod
    def credit_risk_assessment(subsidiaries):
        credit_risks = [sub.financial_metrics['credit_risk_score'] for sub in subsidiaries]
        return {
            'average_credit_risk': np.mean(credit_risks),
            'credit_risk_variance': np.var(credit_risks)
        }

In [11]:
network_analyzer = SubsidiaryNetworkAnalyzer()

parent_company = Company("Acme Corp", "ACME")
network_analyzer.add_company(parent_company)

subsidiaries = [
    Subsidiary("Acme Tech", parent_company),
    Subsidiary("Acme Finance", parent_company),
    Subsidiary("Acme Global", parent_company)
]

for subsidiary in subsidiaries:
    subsidiary.financial_metrics = {
        'annual_revenue': 50000000,
        'credit_risk_score': 0.3,
        'fx_exposure': 0.2,
        'conversion_likelihood': 0
    }
    network_analyzer.add_subsidiary(subsidiary, parent_company.name)

high_potential_subsidiaries = network_analyzer.identify_high_potential_subsidiaries()
print("Top Potential Subsidiaries:", high_potential_subsidiaries)

risk_analyzer = RiskAnalyzer()
fx_risk = risk_analyzer.calculate_fx_risk(subsidiaries)
credit_risk = risk_analyzer.credit_risk_assessment(subsidiaries)

print("FX Risk:", fx_risk)
print("Credit Risk:", credit_risk)

Top Potential Subsidiaries: [('Acme Tech', 15.436666666666667), ('Acme Finance', 15.436666666666667), ('Acme Global', 15.436666666666667), ('Acme Corp', 0.7)]
FX Risk: {'average_fx_exposure': 0.20000000000000004, 'fx_risk_volatility': 2.7755575615628914e-17}
Credit Risk: {'average_credit_risk': 0.3, 'credit_risk_variance': 0.0}


In [14]:
import networkx as nx
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

class EnhancedSubsidiaryNetworkVisualizer:
    def __init__(self):
        self.graph = nx.DiGraph()

    def create_comprehensive_network(self, companies_data):
        """
        Create a network with detailed financial metrics
        """
        for company in companies_data:
            # Add parent company node
            self.graph.add_node(
                company['name'],
                type='parent',
                metrics={
                    'credit_score': company.get('credit_score', 0),
                    'total_revenue': company.get('revenue', 0),
                    'market_cap': company.get('market_cap', 0),
                    'fx_exposure': company.get('fx_exposure', 0),
                    'trading_volume': company.get('trading_volume', 0)
                }
            )

            # Add subsidiaries
            for subsidiary in company.get('subsidiaries', []):
                subsidiary_name = f"{company['name']} - {subsidiary['name']}"
                self.graph.add_node(
                    subsidiary_name,
                    type='subsidiary',
                    metrics={
                        'credit_score': subsidiary.get('credit_score', 0),
                        'revenue': subsidiary.get('revenue', 0),
                        'fx_exposure': subsidiary.get('fx_exposure', 0),
                        'trading_volume': subsidiary.get('trading_volume', 0),
                        'profit_margin': subsidiary.get('profit_margin', 0),
                        'debt_ratio': subsidiary.get('debt_ratio', 0)
                    }
                )

                # Add edge between parent and subsidiary
                self.graph.add_edge(
                    company['name'],
                    subsidiary_name,
                    weight=subsidiary.get('strategic_importance', 1)
                )

    def create_advanced_network_visualization(self):
        """
        Create an advanced network visualization with detailed metrics
        """
        # Compute layout
        pos = nx.spring_layout(self.graph, k=0.5, iterations=50)

        # Prepare node traces
        node_x, node_y = [], []
        node_text = []
        node_sizes = []
        node_colors = []

        # Prepare edge traces
        edge_x, edge_y = [], []

        # Process nodes
        for node in self.graph.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)

            # Prepare hover text with detailed metrics
            metrics = self.graph.nodes[node].get('metrics', {})
            hover_text = f"<b>{node}</b><br>"
            hover_text += "<br>".join([
                f"{k.replace('_', ' ').title()}: {v}"
                for k, v in metrics.items()
            ])
            node_text.append(hover_text)

            # Determine node size and color based on metrics
            node_type = self.graph.nodes[node].get('type', 'unknown')
            if node_type == 'parent':
                node_sizes.append(500)  # Larger for parent companies
                node_colors.append('rgba(255,0,0,0.7)')  # Red for parents
            else:
                node_sizes.append(300)  # Smaller for subsidiaries
                node_colors.append('rgba(0,0,255,0.7)')  # Blue for subsidiaries

        # Process edges
        for edge in self.graph.edges():
            start_pos = pos[edge[0]]
            end_pos = pos[edge[1]]

            edge_x.extend([start_pos[0], end_pos[0], None])
            edge_y.extend([start_pos[1], end_pos[1], None])

        # Create edge trace
        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=0.5, color='#888'),
            hoverinfo='none',
            mode='lines'
        )

        # Create node trace
        node_trace = go.Scatter(
            x=node_x, y=node_y,
            mode='markers+text',
            hoverinfo='text',
            text=node_text,
            marker=dict(
                showscale=True,
                colorscale='Viridis',
                size=node_sizes,
                color=node_colors,
                line_width=2,
                line_color='#000'
            )
        )

        # Create layout
        layout = go.Layout(
            title='Comprehensive Subsidiary Network',
            titlefont_size=16,
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20,l=5,r=5,t=40),
            annotations=[
                dict(
                    text="Node size and color represent company type",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002
                )
            ],
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
        )

        # Create figure
        fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
        return fig

    def generate_network_metrics_summary(self):
        """
        Generate comprehensive network metrics
        """
        metrics_summary = {
            'total_nodes': self.graph.number_of_nodes(),
            'total_edges': self.graph.number_of_edges(),
            'network_density': nx.density(self.graph),
            'node_metrics': {}
        }

        # Aggregate metrics for each node
        for node in self.graph.nodes():
            node_metrics = self.graph.nodes[node].get('metrics', {})
            metrics_summary['node_metrics'][node] = {
                'type': self.graph.nodes[node].get('type', 'unknown'),
                **node_metrics
            }

        return metrics_summary

# Example Usage
def main():
    # Sample comprehensive companies data
    companies_data = [
        {
            'name': 'Global Innovations Inc.',
            'revenue': 1000000000,
            'credit_score': 85,
            'market_cap': 5000000000,
            'fx_exposure': 0.15,
            'trading_volume': 2500000,
            'subsidiaries': [
                {
                    'name': 'Tech Solutions',
                    'revenue': 250000000,
                    'credit_score': 78,
                    'fx_exposure': 0.1,
                    'trading_volume': 500000,
                    'profit_margin': 0.22,
                    'debt_ratio': 0.35,
                    'strategic_importance': 0.8
                },
                {
                    'name': 'Global Finance',
                    'revenue': 150000000,
                    'credit_score': 82,
                    'fx_exposure': 0.25,
                    'trading_volume': 350000,
                    'profit_margin': 0.18,
                    'debt_ratio': 0.45,
                    'strategic_importance': 0.6
                }
            ]
        },
        {
            'name': 'Quantum Enterprises',
            'revenue': 750000000,
            'credit_score': 79,
            'market_cap': 3500000000,
            'fx_exposure': 0.2,
            'trading_volume': 1800000,
            'subsidiaries': [
                {
                    'name': 'Quantum Research',
                    'revenue': 180000000,
                    'credit_score': 75,
                    'fx_exposure': 0.05,
                    'trading_volume': 250000,
                    'profit_margin': 0.15,
                    'debt_ratio': 0.3,
                    'strategic_importance': 0.7
                }
            ]
        }
    ]

    # Initialize and create network
    network_visualizer = EnhancedSubsidiaryNetworkVisualizer()

    # Create comprehensive network
    network_visualizer.create_comprehensive_network(companies_data)

    # Generate visualization
    network_fig = network_visualizer.create_advanced_network_visualization()

    # Generate network metrics summary
    network_metrics = network_visualizer.generate_network_metrics_summary()

    # Display visualization
    network_fig.show()

    # Print network metrics
    print("Network Metrics Summary:")
    print(f"Total Nodes: {network_metrics['total_nodes']}")
    print(f"Total Edges: {network_metrics['total_edges']}")
    print(f"Network Density: {network_metrics['network_density']:.4f}")

    # Print detailed node metrics
    print("\nDetailed Node Metrics:")
    for node, metrics in network_metrics['node_metrics'].items():
        print(f"\n{node}:")
        for metric, value in metrics.items():
            print(f"  {metric.replace('_', ' ').title()}: {value}")

if __name__ == "__main__":
    main()

Network Metrics Summary:
Total Nodes: 5
Total Edges: 3
Network Density: 0.1500

Detailed Node Metrics:

Global Innovations Inc.:
  Type: parent
  Credit Score: 85
  Total Revenue: 1000000000
  Market Cap: 5000000000
  Fx Exposure: 0.15
  Trading Volume: 2500000

Global Innovations Inc. - Tech Solutions:
  Type: subsidiary
  Credit Score: 78
  Revenue: 250000000
  Fx Exposure: 0.1
  Trading Volume: 500000
  Profit Margin: 0.22
  Debt Ratio: 0.35

Global Innovations Inc. - Global Finance:
  Type: subsidiary
  Credit Score: 82
  Revenue: 150000000
  Fx Exposure: 0.25
  Trading Volume: 350000
  Profit Margin: 0.18
  Debt Ratio: 0.45

Quantum Enterprises:
  Type: parent
  Credit Score: 79
  Total Revenue: 750000000
  Market Cap: 3500000000
  Fx Exposure: 0.2
  Trading Volume: 1800000

Quantum Enterprises - Quantum Research:
  Type: subsidiary
  Credit Score: 75
  Revenue: 180000000
  Fx Exposure: 0.05
  Trading Volume: 250000
  Profit Margin: 0.15
  Debt Ratio: 0.3
